In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2-0.2
    resonator_length_2.append(a)
print(resonator_length_2)


[7.253254683870284, 7.024788339841421, 6.80834654020913]


# Flipchip

In [10]:
#V10부터 pad gap 바뀜
#V11 pad gap 바뀌면서 coupler&pad사이 거리 바뀐거 compensation -> big g
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip',
                                            pad_gap='20um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+75}um",
                                              pad_width="110um",
                                              pad_height="504um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [11]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [12]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [13]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [14]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [15]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [16]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [17]:
hfss.start()

INFO 04:03PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:03PM [load_ansys_project]: 	Opened Ansys App
INFO 04:03PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 04:03PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project20
INFO 04:03PM [connect_design]: No active design found (or error getting active design).
INFO 04:03PM [connect]: 	 Connected to project "Project20". No design detected


True

In [18]:
hfss.activate_ansys_design("flip_V11", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

04:03PM 18s WARNING [activate_ansys_design]: The design_name=flip_V11 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 04:03PM [connect_design]: 	Opened active design
	Design:    flip_V11 [Solution type: Eigenmode]
WARNING 04:03PM [connect_setup]: 	No design setup detected.
WARNING 04:03PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:03PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [19]:
hfss.close()

True

# LOM Analysis

In [20]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [21]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.52',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.52',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [23]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 10:21PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpphw7p91_.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5520000000, Maxwell, 1, False
INFO 10:21PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpv75wq5pr.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5520000000, Maxwell, 1, False
INFO 10:21PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpoi9sdmn1.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5520000000, Maxwell, 2, False
INFO 10:21PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpy_2rshx2.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5520000000, Maxwell, 3, False
INFO 10:21PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpgp_cnihq.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5520000000, Maxwell, 4, False
INFO 10:21

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,383.40670,-72.54262,-52.45710,-198.49460
pad_bot_Q0,-72.54262,111.86540,-37.33993,-0.70778
pad_top_Q0,-52.45710,-37.33993,111.71140,-20.79991
quad_coupler_c0,-198.49460,-0.70778,-20.79991,221.05350


In [24]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.86
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.237155 [GHz]
EC 254.982323 [MHz]
EJ 14.854145 [GHz]
alpha -289.185945 [MHz]
dispersion 2.514694 [KHz]
Lq 10.995564 [nH]
Cq 75.966945 [fF]
T1 45.450863 [us]

**Coupling Properties**

tCqbus1 -10.045653 [fF]
gbus1_in_MHz -133.909626 [MHz]
χ_bus1 -3.415009 [MHz]
1/T1bus1 3501.692447 [Hz]
T1bus1 45.450863 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.556072,289.014973,14.854145,-331.154634,8.940677,[-3.525103390426821],[-0.003915061573826021],0.003915,3.525103
2,5.482643,280.965751,14.854145,-321.161428,6.76552,[-203.514280754194],[-11.551028250886604],11.551028,203.514281
3,5.431292,275.412863,14.854145,-314.291684,5.541861,[-195.04522944959803],[-9.765121580908316],9.765122,195.045229
4,5.444819,276.869543,14.854145,-316.0919,5.843069,[-177.6649244953123],[-8.2798574433765],8.279857,177.664924
5,5.388284,270.810365,14.854145,-308.612684,4.675178,[-157.6795262597514],[-5.961871963620875],5.961872,157.679526
6,5.36449,268.282679,14.854145,-305.499552,4.25028,[-150.41894225137705],[-5.227511205491974],5.227511,150.418942
7,5.304467,261.965851,14.854145,-297.737526,3.329356,[-140.4205409553763],[-4.154268877035112],4.154269,140.420541
8,5.282197,259.643577,14.854145,-294.89036,3.036591,[-137.55385075254588],[-3.8543086509014106],3.854309,137.553851
9,5.247712,256.07054,14.854145,-290.516457,2.629109,[-135.24828880087105],[-3.5386767092040383],3.538677,135.248289
10,5.237155,254.982323,14.854145,-289.185945,2.514694,[-133.90962560677963],[-3.41500906582076],3.415009,133.909626


In [25]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
